In [7]:
from __future__ import annotations

from mpi4py import MPI
from petsc4py import PETSc

from pathlib import Path

import dolfinx
import ufl

from dolfinx import default_scalar_type, plot
from tqdm import tqdm
import dolfinx.la as _la
from dolfinx.mesh import CellType, create_rectangle, locate_entities_boundary
from ufl import grad, inner, split
from basix.ufl import blocked_element, element, enriched_element, mixed_element
from dolfinx.fem import Function, functionspace, dirichletbc, Expression, locate_dofs_topological, Constant
import dolfinx_mpc
from dolfinx_mpc import MultiPointConstraint
import numpy as np

import pyvista

In [5]:
nex = int(64)
ney = int(32)

PSRI_control = True
plot_control = True

ele_dict = {0 : "tri_P1",
            1 : "tri_P2",
            2 : "tri_P2B3",
            3 : "qua_P1",
            4 : "qua_P2",
            5 : "qua_S2",}

ele_index = 4
ele_type = ele_dict[ele_index]
tol = 250 * np.finfo(default_scalar_type).resolution

if ele_type == "tri_P2B3":
    cell_type = CellType.triangle
    
elif ele_type == "tri_P2":
    cell_type = CellType.triangle

elif ele_type == "tri_P1":
    cell_type = CellType.triangle
    
elif ele_type == "qua_P2":
    cell_type = CellType.quadrilateral
    
elif ele_type == "qua_P1":
    cell_type = CellType.quadrilateral

elif ele_type == "qua_S2":
    cell_type = CellType.quadrilateral
    

if PSRI_control:
    results_folder = Path(f"results/nonlinear-naghdi/PSRI/cylinder-periodic-arclength/{nex}_{ney}_{ele_type}")
else:
    results_folder = Path(f"results/nonlinear-naghdi/FI/cylinder-periodic-arclength/{nex}_{ney}_{ele_type}")

results_folder.mkdir(exist_ok=True, parents=True)

# Initial shape

In [ ]:
r = 1.016 
L = 3.048 
E, nu = 2.0685E7, 0.3 
mu = E/(2.0*(1.0 + nu)) 
lmbda = 2.0*mu*nu/(1.0 - 2.0*nu) 
t = 0.03 

mesh = create_rectangle(MPI.COMM_WORLD, np.array([[-np.pi/2, 0], [3*np.pi/2, L]]), 
                        [nex, ney], cell_type)

tdim = mesh.topology.dim
fdim = tdim - 1

cell = mesh.basix_cell()
P1 = element("Lagrange", cell, degree=1)
P2 = element("Lagrange", cell, degree=2)
if ele_index == 5:
    S2 = element("Serendipity", cell, degree=2)
elif ele_index == 2:
    B3 = element("Bubble", cell, degree=3)
    
x = ufl.SpatialCoordinate(mesh)
phi0_ufl = ufl.as_vector([r * ufl.sin(x[0]), x[1], r * ufl.cos(x[0])])

def unit_normal(phi):
    n = ufl.cross(phi.dx(0), phi.dx(1))
    return n/ufl.sqrt(inner(n, n))

n0_ufl = unit_normal(phi0_ufl)

def tangent_1(phi):
    t1 = phi.dx(0)
    t1 = t1/ufl.sqrt(inner(t1, t1))
    return t1

def tangent_2(n, t1):
    t2 = ufl.cross(n, t1)
    t2 = t2/ufl.sqrt(inner(t2, t2))
    return t2

t1_ufl = tangent_1(phi0_ufl)
t2_ufl = tangent_2(n0_ufl, t1_ufl)

# the analytical expression of R0
def rotation_matrix(t1, t2, n):
    R = ufl.as_matrix([[t1[0], t2[0], n[0]], 
                       [t1[1], t2[1], n[1]], 
                       [t1[2], t2[2], n[2]]])
    return R

R0_ufl = rotation_matrix(t1_ufl, t2_ufl, n0_ufl)

# Update the director with two successive elementary rotations
def director(R0, theta):
    Lm3 = ufl.as_vector([ufl.sin(theta[1])*ufl.cos(theta[0]), -ufl.sin(theta[0]), ufl.cos(theta[1])*ufl.cos(theta[0])])
    d = ufl.dot(R0, Lm3)
    return d

if plot_control:
    P1_d3_FS = functionspace(mesh, blocked_element(P1, shape = (3,)))

    n0_P1_expr = Expression(n0_ufl, P1_d3_FS.element.interpolation_points())
    n0_P1_func = Function(P1_d3_FS)
    n0_P1_func.interpolate(n0_P1_expr)

    phi0_P1_expr = Expression(phi0_ufl, P1_d3_FS.element.interpolation_points())
    phi0_P1_func = Function(P1_d3_FS)
    phi0_P1_func.interpolate(phi0_P1_expr)

    topology, cell_types, geometry = plot.vtk_mesh(P1_d3_FS)

    geometry_phi0_P1 = phi0_P1_func.x.array.reshape((geometry.shape[0], len(phi0_P1_func)))
    geometry_n0_P1 = n0_P1_func.x.array.reshape((geometry.shape[0], len(n0_P1_func)))

    grid_phi0_P1 = pyvista.UnstructuredGrid(topology, cell_types, geometry_phi0_P1)
    grid_phi0_P1["n0"] = geometry_n0_P1
    glyphs = grid_phi0_P1.glyph(orient="n0", factor=0.1)


    plotter = pyvista.Plotter(off_screen = True)
    plotter.add_mesh(grid_phi0_P1, style="wireframe", color="k")
    plotter.add_mesh(glyphs,  show_scalar_bar=True, scalar_bar_args={"vertical": True})
    plotter.show_grid()
    plotter.show_axes_all()
    plotter.enable_parallel_projection()
    plotter.show()
    plotter.close()

# Shell model

In [8]:
if ele_type == "tri_P2B3":
    P2B3 = enriched_element([P2, B3])
    naghdi_shell_element = mixed_element(
        [blocked_element(P2B3, shape=(3,)), blocked_element(P2, shape=(2,))]
    )
elif ele_type == "tri_P2":
    naghdi_shell_element = mixed_element(
        [blocked_element(P2, shape=(3,)), blocked_element(P2, shape=(2,))]
        )

elif ele_type == "tri_P1":
    naghdi_shell_element = mixed_element(
        [blocked_element(P1, shape=(3,)), blocked_element(P1, shape=(2,))]
    )
    
elif ele_type == "qua_P2":
    naghdi_shell_element = mixed_element(
        [blocked_element(P2, shape=(3,)), blocked_element(P2, shape=(2,))]
    )
elif ele_type == "qua_P1":
    naghdi_shell_element = mixed_element(
        [blocked_element(P1, shape=(3,)), blocked_element(P1, shape=(2,))]
    )

elif ele_type == "qua_S2":
    naghdi_shell_element = mixed_element(
        [blocked_element(S2, shape=(3,)), blocked_element(S2, shape=(2,))]
    )
    
naghdi_shell_FS = functionspace(mesh, naghdi_shell_element)

q_func = Function(naghdi_shell_FS) # current configuration
q_trial = ufl.TrialFunction(naghdi_shell_FS)
q_test = ufl.TestFunction(naghdi_shell_FS)

u_func, theta_func = split(q_func) # current displacement and rotation

# current deformation gradient 
F = grad(u_func) + grad(phi0_ufl) 

# current director
d = director(R0_ufl, theta_func)

# initial metric and curvature tensor a0 and b0
a0_ufl = grad(phi0_ufl).T * grad(phi0_ufl)
b0_ufl = 0.5*( grad(phi0_ufl).T * grad(n0_ufl) + grad(n0_ufl).T * grad(phi0_ufl))

def epsilon(F):
    """Membrane strain"""
    return 0.5 * (F.T * F - a0_ufl)


def kappa(F, d):
    """Bending strain"""
    return 0.5 * (F.T * grad(d) + grad(d).T * F) - b0_ufl


def gamma(F, d):
    """Transverse shear strain"""
    return F.T * d

a0_contra_ufl = ufl.inv(a0_ufl)
j0_ufl = ufl.det(a0_ufl)

i,j,l,m = ufl.indices(4)  # noqa: E741
A_contra_ufl = ufl.as_tensor( ( ((2.0*lmbda*mu) / (lmbda + 2.0*mu)) * a0_contra_ufl[i,j]*a0_contra_ufl[l,m]
                + 1.0*mu* (a0_contra_ufl[i,l]*a0_contra_ufl[j,m] + a0_contra_ufl[i,m]*a0_contra_ufl[j,l]) )
                ,[i,j,l,m])

N = ufl.as_tensor(t * A_contra_ufl[i,j,l,m] * epsilon(F)[l,m], [i,j])

M = ufl.as_tensor( (t**3 / 12.0) * A_contra_ufl[i,j,l,m]*kappa(F, d)[l,m], [i,j])

T = ufl.as_tensor( (t * mu *5.0 / 6.0) * a0_contra_ufl[i, j] * gamma(F, d)[j], [i])

psi_m = 0.5*inner(N, epsilon(F))

psi_b = 0.5*inner(M, kappa(F, d))

psi_s = 0.5*inner(T, gamma(F, d))

if ele_type == "qua_P1" or ele_type == "tri_P1":
    dx_f = ufl.Measure('dx', domain=mesh, metadata={"quadrature_degree": 2})
    if PSRI_control:
        dx_r = ufl.Measure('dx', domain=mesh, metadata={"quadrature_degree": 1})
    else:
        dx_r = ufl.Measure('dx', domain=mesh, metadata={"quadrature_degree": 2})
    
else:
    dx_f = ufl.Measure('dx', domain=mesh, metadata={"quadrature_degree": 4})
    if PSRI_control:
        dx_r = ufl.Measure('dx', domain=mesh, metadata={"quadrature_degree": 2})
    else:
        dx_r = ufl.Measure('dx', domain=mesh, metadata={"quadrature_degree": 4})


# Calculate the factor alpha as a function of the mesh size h
h = ufl.CellDiameter(mesh)
alpha_FS = functionspace(mesh, element("DG", cell, 0))
alpha_expr = Expression(t**2 / h**2, alpha_FS.element.interpolation_points())
alpha = Function(alpha_FS)
alpha.interpolate(alpha_expr)

# Full integration part of the total elastic energy
Pi_PSRI = psi_b * ufl.sqrt(j0_ufl) * dx_f 
Pi_PSRI += alpha * psi_m * ufl.sqrt(j0_ufl) * dx_f
Pi_PSRI += alpha * psi_s * ufl.sqrt(j0_ufl) * dx_f

# Reduced integration part of the total elastic energy
Pi_PSRI += (1.0 - alpha) * psi_m * ufl.sqrt(j0_ufl) * dx_r
Pi_PSRI += (1.0 - alpha) * psi_s * ufl.sqrt(j0_ufl) * dx_r

# external work part (zero in this case)
f = Constant(mesh, default_scalar_type((0.0, 0.0, 0.0, 0.0, 0.0)))
Wext = ufl.inner(f, q_func)*dx_f

Fint = ufl.derivative(Pi_PSRI, q_func, q_test)
Fext = ufl.derivative(Wext, q_func, q_test)

Residual = Fint - Fext
Jacobian = ufl.derivative(Residual, q_func, q_trial)

# Boundary Conditions

In [9]:
# Dirichlet boundary conditions
def clamped_boundary(x):
    return np.isclose(x[1], 0.0, atol = tol)

clamped_facets = locate_entities_boundary(mesh, fdim, clamped_boundary)

u_FS, _ = naghdi_shell_FS.sub(0).collapse()
theta_FS, _ = naghdi_shell_FS.sub(1).collapse()

# u1, u2, u3 = 0 on the clamped boundary
u_clamped = Function(u_FS) # default value is 0
clamped_dofs_u = locate_dofs_topological((naghdi_shell_FS.sub(0), u_FS), fdim, clamped_facets)
bc_clamped_u = dirichletbc(u_clamped, clamped_dofs_u, naghdi_shell_FS.sub(0))

# theta1, theta2 = 0 on the clamped boundary
theta_clamped = Function(theta_FS) # default value is 0
clamped_dofs_theta = locate_dofs_topological((naghdi_shell_FS.sub(1), theta_FS), fdim, clamped_facets)
bc_clamped_theta = dirichletbc(theta_clamped, clamped_dofs_theta, naghdi_shell_FS.sub(1))

bcs = [bc_clamped_u, bc_clamped_theta]

# Create MPC
def periodic_boundary(x):
    return np.isclose(x[0], 3*np.pi/2, atol=tol)

def periodic_relation(x):
    out_x = np.zeros_like(x)
    out_x[0] = x[0] - 2*np.pi
    out_x[1] = x[1]
    out_x[2] = x[2]
    return out_x

mpc = MultiPointConstraint(naghdi_shell_FS)
mpc.create_periodic_constraint_geometrical(naghdi_shell_FS.sub(0).sub(0), periodic_boundary, periodic_relation, bcs)
mpc.create_periodic_constraint_geometrical(naghdi_shell_FS.sub(0).sub(1), periodic_boundary, periodic_relation, bcs)
mpc.create_periodic_constraint_geometrical(naghdi_shell_FS.sub(0).sub(2), periodic_boundary, periodic_relation, bcs)
mpc.create_periodic_constraint_geometrical(naghdi_shell_FS.sub(1).sub(0), periodic_boundary, periodic_relation, bcs)
mpc.create_periodic_constraint_geometrical(naghdi_shell_FS.sub(1).sub(1), periodic_boundary, periodic_relation, bcs)
mpc.finalize()

# Check the MPC
num_slaves_global = mesh.comm.allreduce(len(mpc.slaves), op=MPI.SUM)
num_masters_global = mesh.comm.allreduce(len(mpc.masters.array), op=MPI.SUM)

assert num_slaves_global > 0
assert num_masters_global == num_slaves_global

print(f"number of total master dofs: {num_masters_global}")
print(f"number of total slave dofs: {num_slaves_global}")

number of total master dofs: 320
number of total slave dofs: 320


# Point source

In [ ]:
def compute_cell_contributions(V, points):
    # Determine what process owns a point and what cells it lies within
    mesh = V.mesh
    _, _, owning_points, cells = dolfinx.cpp.geometry.determine_point_ownership(
        mesh._cpp_object, points, 1e-6)
    owning_points = np.asarray(owning_points).reshape(-1, 3)
    # Pull owning points back to reference cell
    mesh_nodes = mesh.geometry.x
    cmap = mesh.geometry.cmap
    ref_x = np.zeros((len(cells), mesh.geometry.dim),
                     dtype=mesh.geometry.x.dtype)
    for i, (point, cell) in enumerate(zip(owning_points, cells)):
        geom_dofs = mesh.geometry.dofmap[cell]
        ref_x[i] = cmap.pull_back(point.reshape(-1, 3), mesh_nodes[geom_dofs])

    # Create expression evaluating a trial function (i.e. just the basis function)
    u = ufl.TrialFunction(V.sub(0).sub(2))
    num_dofs = V.sub(0).sub(2).dofmap.dof_layout.num_dofs * V.sub(0).sub(2).dofmap.bs
    if len(cells) > 0:
        # NOTE: Expression lives on only this communicator rank
        expr = dolfinx.fem.Expression(u, ref_x, comm=MPI.COMM_SELF)
        values = expr.eval(mesh, np.asarray(cells, dtype=np.int32))
        print(f"values: {values}")
        # Strip out basis function values per cell
        basis_values = values[:num_dofs:num_dofs*len(cells)]
    else:
        basis_values = np.zeros(
            (0, num_dofs), dtype=dolfinx.default_scalar_type)
    return cells, basis_values

if mesh.comm.rank == 0:
    points_A = np.array([[0.0, L, 0.0]], dtype=mesh.geometry.x.dtype)
    points_B = np.array([[np.pi, L, 0.0]], dtype=mesh.geometry.x.dtype)
else:
    points_A = np.zeros((0, 3), dtype=mesh.geometry.x.dtype)
    points_B = np.zeros((0, 3), dtype=mesh.geometry.x.dtype)

ps_cells_A, basis_values_A = compute_cell_contributions(naghdi_shell_FS, points_A)
ps_cells_B, basis_values_B = compute_cell_contributions(naghdi_shell_FS, points_B)

print(f"cells_A: {ps_cells_A}")
print(f"basis_values_A: {basis_values_A}")
print(f"cells_B: {ps_cells_B}")
print(f"basis_values_B: {basis_values_B}")